## 다음 랭킹 뉴스

In [4]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://news.daum.net/ranking/popular?regDate=20210118" 
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html5lib")

    다음 뉴스의 랭킹 뉴스는 "많이 본", "열독률 높은", "댓글 많은", "연령별" 로 나뉘어져 있다. 뉴스 url에서 tag로 나뉘어져 있는데 https://news.daum.net/ranking/ 뒤에 각각 "popular", "kkomkkom", "bestreply", "age"로 구분지어져 있다. 각각의 tag의 뉴스 구성은 전혀 다르게 되어 있어 따로 크롤링을 해야 한다. 그 외에도 연예, 스포츠 뉴스를 따로 구분하나 지금 당장 필요하진 않으므로 굳이 크롤링 해오진 않을 계획이다.

In [9]:
# 뉴스 태그 구분
tag = ["popular", "kkomkkom", "bestreply", "age"]

In [185]:
# demo v0.1 popular tag 뉴스만 가져오기
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://news.daum.net/ranking/popular?regDate=20210118" 
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html5lib")

# 랭크, 언론사, URL, 제목 가져오기
ranking_box = soup.find_all(class_ = "rank_num rank_popular")
contents_box = soup.find_all(class_ = "cont_thumb")
l = []

for num in range(50):
    d = {}
    d['Rank'] = ranking_box[num].find(class_ = "screen_out").get_text()
    
    new_info = contents_box[num].find(class_ ="tit_thumb")
    d['Press'] = new_info.find(class_ = 'info_news').get_text()
    d['URL'] = new_info.find('a')['href']
    d['Title'] = new_info.find('a').get_text()
    l.append(d)

# 기사 요약본, 메인 텍스트 가져오기
for link in l:
    resp = requests.get(link['URL'])
    soup = BeautifulSoup(resp.text, "html5lib")
    contents = soup.find(class_ = "news_view")
    link['Main Article'] = contents.find(class_ = "article_view").get_text()
    
df = pd.DataFrame(l)

In [184]:
# demo v0.2 popular, kkomkkom, bestreply, age tag 뉴스만 가져오기
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# 뉴스 태그 구분
# tag = ["popular", "kkomkkom", "bestreply", "age"]
tag = ["popular", "kkomkkom", "bestreply", "age"]


def popular(soup):
    # 랭크, 언론사, URL, 제목 가져오기
    ranking_box = soup.find_all(class_="rank_num rank_popular")
    contents_box = soup.find_all(class_="cont_thumb")
    l = []

    for num in range(50):
        d = {}
        d['Rank'] = ranking_box[num].find(class_="screen_out").get_text()

        new_info = contents_box[num].find(class_="tit_thumb")
        d['Press'] = new_info.find(class_='info_news').get_text()
        d['URL'] = new_info.find('a')['href']
        d['Title'] = new_info.find('a').get_text()
        l.append(d)

    # 기사 요약본, 메인 텍스트 가져오기
    for link in l:
        resp = requests.get(link['URL'])
        soup = BeautifulSoup(resp.text, "html5lib")
        contents = soup.find(class_="news_view")
        link['Main Article'] = contents.find(class_="article_view").get_text()

    return pd.DataFrame(l)


def kkomkkom(soup):
    # 랭크, 언론사, URL, 제목 가져오기
    ranking_box = soup.find_all(class_="rank_num rank_popular")
    contents_box = soup.find_all(class_="cont_thumb")
    l = []

    for num in range(30):
        d = {}
        d['Rank'] = ranking_box[num].find(class_="screen_out").get_text()

        new_info = contents_box[num].find(class_="tit_thumb")
        d['Press'] = new_info.find(class_='info_news').get_text()
        d['URL'] = new_info.find('a')['href']
        d['Title'] = new_info.find('a').get_text()
        l.append(d)

    # 기사 요약본, 메인 텍스트 가져오기
    for link in l:
        resp = requests.get(link['URL'])
        soup = BeautifulSoup(resp.text, "html5lib")
        contents = soup.find(class_="news_view")
        link['Main Article'] = contents.find(class_="article_view").get_text()

    return pd.DataFrame(l)


def bestreply(soup):
    # 랭크, 언론사, URL, 제목 가져오기
    ranking_box = soup.find_all(class_="rank_num")
    contents_box = soup.find_all(class_="cont_thumb")
    l = []

    for num in range(30):
        d = {}
        d['Rank'] = ranking_box[num].find(class_="screen_out").get_text()
        d['Comment'] = ranking_box[num].find(class_="ico_news2").get_text()

        new_info = contents_box[num].find(class_="tit_thumb")
        d['Press'] = new_info.find(class_='info_news').get_text()
        d['URL'] = new_info.find('a')['href']
        d['Title'] = new_info.find('a').get_text()
        l.append(d)

    # 기사 요약본, 메인 텍스트 가져오기
    for link in l:
        resp = requests.get(link['URL'])
        soup = BeautifulSoup(resp.text, "html5lib")
        contents = soup.find(class_="news_view")
        link['Main Article'] = contents.find(class_="article_view").get_text()

    return pd.DataFrame(l)


def age(soup):
    # 여성과 남성의 ranking news 모으기
    female = soup.find_all(class_="rank_female")
    male = soup.find_all(class_="rank_male")
    ranking_news = [female, male]

    df = pd.DataFrame()
    for news in ranking_news:     # female -> male
        for i in range(4):        # 20s -> 50s
            l = []
            age = news[i].find(class_="txt_news").get_text()[:3]
            sex = news[i].find(class_="txt_news").get_text()[4:]
            press_list = news[i].find_all(class_="info_news")
            news_list = news[i].find_all(class_="link_txt")

            for num in range(5):
                d = {}
                d['Age'] = age
                d['Sex'] = sex
                d['Rank'] = num + 1
                d['Press'] = press_list[num].get_text()
                d['URL'] = news_list[num]['href']
                d['Title'] = news_list[num].get_text()
                l.append(d)

            for link in l:
                resp = requests.get(link['URL'])
                soup = BeautifulSoup(resp.text, "html5lib")
                contents = soup.find(class_="news_view")
                link['Main Article'] = contents.find(
                    class_="article_view").get_text()

            df = pd.concat([df, pd.DataFrame(l)], axis=0, ignore_index=True)

    return df


url = "https://news.daum.net/ranking/age?regDate=20210118"
resp = requests.get(url)
soup = BeautifulSoup(resp.text, "html5lib")
# popular = popular(soup)
# kkomkkom = kkomkkom(soup)
# bestreply = bestreply(soup)
# age = age(soup)

In [1]:
# demo v0.3 날짜 입력을 받아 랭킹 뉴스 자동으로 긁어오기
import os
os.chdir(r"C:/Users/cjy89/NLP/Project_news_crawling/")
tags = ["popular", "kkomkkom", "bestreply", "age"]

import re
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

def popular(soup, date):
    # 랭크, 언론사, URL, 제목 가져오기
    ranking_box = soup.find_all(class_="rank_num rank_popular")
    contents_box = soup.find_all(class_="cont_thumb")
    l = []

    for num in range(50):
        d = {}
        d['Date'] = int(date)
        d['Rank'] = ranking_box[num].find(class_="screen_out").get_text()

        new_info = contents_box[num].find(class_="tit_thumb")
        d['Press'] = new_info.find(class_='info_news').get_text()
        d['URL'] = new_info.find('a')['href']
        d['Title'] = new_info.find('a').get_text()
        l.append(d)

    # 기사 요약본, 메인 텍스트 가져오기
    for link in l:
        resp = requests.get(link['URL'])
        soup = BeautifulSoup(resp.text, "html5lib")
        contents = soup.find(class_="news_view")
        link['Main Article'] = contents.find(class_="article_view").get_text()

    return pd.DataFrame(l)


def kkomkkom(soup, date):
    # 랭크, 언론사, URL, 제목 가져오기
    ranking_box = soup.find_all(class_="rank_num rank_popular")
    contents_box = soup.find_all(class_="cont_thumb")
    l = []

    for num in range(30):
        d = {}
        d['Date'] = int(date)
        d['Rank'] = ranking_box[num].find(class_="screen_out").get_text()

        new_info = contents_box[num].find(class_="tit_thumb")
        d['Press'] = new_info.find(class_='info_news').get_text()
        d['URL'] = new_info.find('a')['href']
        d['Title'] = new_info.find('a').get_text()
        l.append(d)

    # 기사 요약본, 메인 텍스트 가져오기
    for link in l:
        resp = requests.get(link['URL'])
        soup = BeautifulSoup(resp.text, "html5lib")
        contents = soup.find(class_="news_view")
        link['Main Article'] = contents.find(class_="article_view").get_text()

    return pd.DataFrame(l)


def bestreply(soup, date):
    # 랭크, 언론사, URL, 제목 가져오기
    ranking_box = soup.find_all(class_="rank_num")
    contents_box = soup.find_all(class_="cont_thumb")
    l = []

    for num in range(30):
        d = {}
        d['Date'] = int(date)
        d['Rank'] = ranking_box[num].find(class_="screen_out").get_text()
        d['Comment'] = ranking_box[num].find(class_="ico_news2").get_text()

        new_info = contents_box[num].find(class_="tit_thumb")
        d['Press'] = new_info.find(class_='info_news').get_text()
        d['URL'] = new_info.find('a')['href']
        d['Title'] = new_info.find('a').get_text()
        l.append(d)

    # 기사 요약본, 메인 텍스트 가져오기
    for link in l:
        resp = requests.get(link['URL'])
        soup = BeautifulSoup(resp.text, "html5lib")
        contents = soup.find(class_="news_view")
        link['Main Article'] = contents.find(class_="article_view").get_text()

    return pd.DataFrame(l)


def age(soup, date):
    # 여성과 남성의 ranking news 모으기
    female = soup.find_all(class_="rank_female")
    male = soup.find_all(class_="rank_male")
    ranking_news = [female, male]

    df = pd.DataFrame()
    for news in ranking_news:     # female -> male
        for i in range(4):        # 20s -> 50s
            l = []
            age = news[i].find(class_="txt_news").get_text()[:3]
            sex = news[i].find(class_="txt_news").get_text()[4:]
            press_list = news[i].find_all(class_="info_news")
            news_list = news[i].find_all(class_="link_txt")

            for num in range(5):
                d = {}
                d['Date'] = int(date)
                d['Age'] = age
                d['Sex'] = sex
                d['Rank'] = num + 1
                d['Press'] = press_list[num].get_text()
                d['URL'] = news_list[num]['href']
                d['Title'] = news_list[num].get_text()
                l.append(d)

            for link in l:
                resp = requests.get(link['URL'])
                soup = BeautifulSoup(resp.text, "html5lib")
                contents = soup.find(class_="news_view")
                link['Main Article'] = contents.find(
                    class_="article_view").get_text()

            df = pd.concat([df, pd.DataFrame(l)], axis=0, ignore_index=True)

    return df


def get_ranking_news(date):
    total_time = 0
    for tag in tags:
        start = time.time()
        url = "https://news.daum.net/ranking/" + tag + "?regDate=" + str(date)
        resp = requests.get(url)
        soup = BeautifulSoup(resp.text, "html5lib")
        
        if(tag == "popular"):
            df = popular(soup, date)
        elif(tag =="kkomkkom"):
            df = kkomkkom(soup, date)
        elif(tag == "bestreply"):
            df = bestreply(soup, date)
        elif(tag == "age"):
            df = age(soup, date)
        
        title = "Daum/" + tag + "/" + str(date) + "_ranking_news.csv"
        df.to_csv(title, sep=",", index=False, encoding="utf-8-sig")
        end = time.time()
        total_time += end-start
        print("Crawling " + str(date) + " " + tag + " news : ", end-start)
    print("Total time :", total_time)
    print("Average time : ", total_time/4)
    print("───────────────────")

In [3]:
for i in range(24):
    get_ranking_news(20210101 + i)

Crawling 20210101 popular news :  14.202004194259644
Crawling 20210101 kkomkkom news :  8.70995044708252
Crawling 20210101 bestreply news :  6.056759595870972
Crawling 20210101 age news :  10.833054780960083
Total time : 39.80176901817322
Average time :  9.950442254543304
───────────────────
Crawling 20210102 popular news :  14.436276912689209
Crawling 20210102 kkomkkom news :  8.053711891174316
Crawling 20210102 bestreply news :  6.168506860733032
Crawling 20210102 age news :  10.931079149246216
Total time : 39.58957481384277
Average time :  9.897393703460693
───────────────────
Crawling 20210103 popular news :  14.312278270721436
Crawling 20210103 kkomkkom news :  8.491061210632324
Crawling 20210103 bestreply news :  6.082691192626953
Crawling 20210103 age news :  10.851634502410889
Total time : 39.7376651763916
Average time :  9.9344162940979
───────────────────
Crawling 20210104 popular news :  13.139715433120728
Crawling 20210104 kkomkkom news :  8.563136577606201
Crawling 2021010

## 다음 기사 분석

In [5]:
# 데이터 불러오기
import os
os.chdir(r"C:/Users/cjy89/NLP/Project_news_crawling/Daum/")